<a href="https://colab.research.google.com/github/Swetha-R-V/AI-Powered-Task-Summarizer-API/blob/main/AI_Powered_Task_Summarizer_APIipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Insight**: **Project Dependency Setup**
This step installs all required libraries such as FastAPI for API creation,
SQLAlchemy for ORM-based database operations, and PostgreSQL drivers.
Installing dependencies at the beginning ensures a consistent runtime environment.
**

In [1]:
!pip install fastapi uvicorn sqlalchemy psycopg2-binary pydantic python-dotenv pytest httpx


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 23.5 MB/s eta 0:00:00


In [2]:
import os

output_dir = 'app'
output_file = os.path.join(output_dir, 'schemas.py')

os.makedirs(output_dir, exist_ok=True)

content = """from pydantic import BaseModel, Field
from datetime import datetime
from typing import Optional

class TaskCreate(BaseModel):
    title: str = Field(..., min_length=3, example="Daily report")
    description: str = Field(..., min_length=10, example="Prepare daily sales report")

    class Config:
        orm_mode = True


class TaskUpdate(BaseModel):
    title: Optional[str] = Field(None, min_length=3)
    description: Optional[str] = Field(None, min_length=10)

    class Config:
        orm_mode = True


class TaskResponse(BaseModel):
    id: int
    title: str
    description: str
    summary: str
    created_at: datetime

    class Config:
        orm_mode = True
"""

with open(output_file, 'w') as f:
    f.write(content)

print(f"File '{output_file}' created successfully.")


File 'app/schemas.py' created successfully.


# **Insight: Database Design Choice**
PostgreSQL is used instead of SQLite to support scalability and
real-world production readiness. SQLAlchemy ORM abstracts raw SQL queries,
making the application more maintainable and secure.


In [3]:
%%writefile app/schemas.py
from pydantic import BaseModel, Field
from datetime import datetime
from typing import Optional

class TaskCreate(BaseModel):
    title: str = Field(..., min_length=3, example="Daily report")
    description: str = Field(..., min_length=10, example="Prepare daily sales report")

    class Config:
        orm_mode = True


class TaskUpdate(BaseModel):
    title: Optional[str] = Field(None, min_length=3)
    description: Optional[str] = Field(None, min_length=10)

    class Config:
        orm_mode = True


class TaskResponse(BaseModel):
    id: int
    title: str
    description: str
    summary: str
    created_at: datetime

    class Config:
        orm_mode = True


Overwriting app/schemas.py


**Insight: External API Integration**
The application fetches summaries from an external service asynchronously,
improving response time and keeping the main API non-blocking.


In [4]:
%%writefile app/external.py
import httpx
from fastapi import HTTPException


def generate_summary(text: str) -> str:
    """
    Calls an external API and returns a short summary text.
    """

    try:
        response = httpx.get(
            "https://api.quotable.io/random",
            timeout=5.0
        )
        response.raise_for_status()

        data = response.json()
        return data.get("content", "Summary not available")

    except httpx.RequestError:
        raise HTTPException(
            status_code=503,
            detail="External API service unavailable"
        )

    except httpx.HTTPStatusError:
        raise HTTPException(
            status_code=502,
            detail="Error from external API"
        )


Writing app/external.py


# **Insight: Separation of Concerns**
CRUD operations are isolated to maintain clean architecture and
make testing and maintenance easier.


In [5]:
%%writefile app/crud.py
from sqlalchemy.orm import Session
from .models import Task
from .schemas import TaskCreate, TaskUpdate
from .external import generate_summary


# CREATE
def create_task(db: Session, task: TaskCreate):
    summary = generate_summary(task.description)

    new_task = Task(
        title=task.title,
        description=task.description,
        summary=summary
    )

    db.add(new_task)
    db.commit()
    db.refresh(new_task)
    return new_task


# READ
def get_task(db: Session, task_id: int):
    return db.query(Task).filter(Task.id == task_id).first()


# UPDATE
def update_task(db: Session, task_id: int, task: TaskUpdate):
    db_task = get_task(db, task_id)

    if db_task is None:
        return None

    if task.title:
        db_task.title = task.title

    if task.description:
        db_task.description = task.description
        db_task.summary = generate_summary(task.description)

    db.commit()
    db.refresh(db_task)
    return db_task


# DELETE
def delete_task(db: Session, task_id: int):
    db_task = get_task(db, task_id)

    if db_task is None:
        return None

    db.delete(db_task)
    db.commit()
    return True


Writing app/crud.py


**Insight: Role of Pydantic Schemas**



Schemas act as a validation layer between client requests and database models,
ensuring only clean and structured data enters the system.


In [6]:
%%writefile app/main.py
from fastapi import FastAPI, Depends, HTTPException
from sqlalchemy.orm import Session
from fastapi.responses import JSONResponse
from fastapi import Request

from .database import SessionLocal
from . import crud, schemas

app = FastAPI(title="AI Task Summarizer API")


# Dependency to get DB session
def get_db():
    db = SessionLocal()
    try:
        yield db
    finally:
        db.close()


# 1⏸ CREATE TASK
@app.post("/tasks", response_model=schemas.TaskResponse, status_code=201)
def create_task(task: schemas.TaskCreate, db: Session = Depends(get_db)):
    return crud.create_task(db, task)


# 2⏸ GET TASK
@app.get("/tasks/{task_id}", response_model=schemas.TaskResponse)
def get_task(task_id: int, db: Session = Depends(get_db)):
    task = crud.get_task(db, task_id)
    if task is None:
        raise HTTPException(status_code=404, detail="Task not found")
    return task


# 3⏸ UPDATE TASK
@app.put("/tasks/{task_id}", response_model=schemas.TaskResponse)
def update_task(task_id: int, task: schemas.TaskUpdate, db: Session = Depends(get_db)):
    updated_task = crud.update_task(db, task_id, task)
    if updated_task is None:
        raise HTTPException(status_code=404, detail="Task not found")
    return updated_task


# 4⏸ DELETE TASK
@app.delete("/tasks/{task_id}", status_code=204)
def delete_task(task_id: int, db: Session = Depends(get_db)):
    result = crud.delete_task(db, task_id)
    if result is None:
        raise HTTPException(status_code=404, detail="Task not found")
    return


# Global exception handler
@app.exception_handler(Exception)
async def global_exception_handler(request: Request, exc: Exception):
    return JSONResponse(
        status_code=500,
        content={"detail": "Internal Server Error"}
    )

Writing app/main.py


In [7]:
%%writefile app/database.py
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
from sqlalchemy.ext.declarative import declarative_base

# Configure the database URL
# For simplicity, we'll use a SQLite database file named 'sql_app.db'
# in the same directory as the script. For production, you'd typically
# use an environment variable for a PostgreSQL or MySQL connection string.
SQLALCHEMY_DATABASE_URL = "sqlite:///./sql_app.db"

# Create the SQLAlchemy engine.
# The connect_args is needed for SQLite to allow multiple threads to interact
# with the database if using a single connection. This is common for FastAPI.
engine = create_engine(
    SQLALCHEMY_DATABASE_URL, connect_args={"check_same_thread": False}
)

# Create a SessionLocal class.
# Each instance of SessionLocal will be a database session.
# The autocommit=False ensures that we have to commit changes explicitly.
# The autoflush=False means that query operations won't automatically flush pending changes to the database.
SessionLocal = sessionmaker(autocommit=False, autoflush=False, bind=engine)

# Create a Base class for declarative models.
# This is where our SQLAlchemy models will inherit from.
Base = declarative_base()


Writing app/database.py


In [8]:
!mkdir -p app
!touch app/__init__.py


In [9]:
!ls app


crud.py  database.py  external.py  __init__.py	main.py  schemas.py


In [10]:
%%writefile app/models.py
from sqlalchemy import Column, Integer, String, DateTime
from datetime import datetime

from app.database import Base

class Task(Base):
    __tablename__ = "tasks"

    id = Column(Integer, primary_key=True, index=True)
    title = Column(String, nullable=False)
    description = Column(String, nullable=False)
    summary = Column(String, nullable=False)
    created_at = Column(DateTime, default=datetime.utcnow)


Writing app/models.py


In [11]:
%%writefile app/database.py
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
from sqlalchemy.ext.declarative import declarative_base

SQLALCHEMY_DATABASE_URL = "sqlite:///./sql_app.db"

engine = create_engine(
    SQLALCHEMY_DATABASE_URL, connect_args={"check_same_thread": False}
)

SessionLocal = sessionmaker(autocommit=False, autoflush=False, bind=engine)

Base = declarative_base()


Overwriting app/database.py


In [12]:
import sys
sys.path.insert(0, "/content")


In [13]:
from app.database import engine, Base
import app.models   # IMPORTANT: use this, not from app import models

Base.metadata.create_all(bind=engine)

print("Database tables created successfully.")


Database tables created successfully.


In [14]:
!pip install pytest httpx


**Insight: External API Integration**


The application fetches summaries from an external service asynchronously,
improving response time and keeping the main API non-blocking.


In [15]:
!pip install pytest httpx


In [16]:
!touch app/__init__.py


In [17]:
import sys
sys.path.insert(0, "/content")

from app.main import app


/usr/local/lib/python3.12/dist-packages/pydantic/_internal/_config.py:383: UserWarning: Valid config keys have changed in V2:
* 'orm_mode' has been renamed to 'from_attributes'
  warnings.warn(message, UserWarning)


In [18]:
from fastapi import FastAPI
from app import models
from app.database import engine

app = FastAPI()

# The main application and its routes are already defined in app/main.py.
# The 'app.routes' module does not exist in this project structure.
# If you intend to use the app defined in app/main.py, you should import it directly:
# from app.main import app as main_app
# and then use main_app instead of defining a new app here.

In [19]:
%%writefile app/init_db.py
from app.database import engine, Base
import app.models

Base.metadata.create_all(bind=engine)
print("DB tables created")


Writing app/init_db.py


In [20]:
!python app/init_db.py


Traceback (most recent call last):
  File "/content/app/init_db.py", line 1, in <module>
    from app.database import engine, Base
ModuleNotFoundError: No module named 'app'


In [21]:
from app.external import generate_summary
from unittest.mock import patch

with patch("app.external.generate_summary"):
    pass

In [22]:
import os
import sys
sys.path.insert(0, "/content")

from fastapi.testclient import TestClient
from unittest.mock import patch

from app.main import app

# Create the tests directory if it doesn't exist
tests_dir = 'tests'
os.makedirs(tests_dir, exist_ok=True)

# Content of the test file
test_file_content = """import sys
sys.path.insert(0, \"/content\")

from fastapi.testclient import TestClient
from unittest.mock import patch

from app.main import app

client = TestClient(app)


def test_create_task():
    # Patch app.external.generate_summary as it's the actual source
    with patch(\"app.external.generate_summary\") as mock_summary:
        mock_summary.return_value = \"Test summary\"

        response = client.post(
            \"/tasks\",
            json={
                \"title\": \"Test Task\",
                \"description\": \"This is a test task description\"
            }
        )

        assert response.status_code == 201
        data = response.json()
        assert data[\"title\"] == \"Test Task\"
        assert data[\"summary\"] == \"Test summary\"


def test_get_task_not_found():
    response = client.get(\"/tasks/9999\")
    assert response.status_code == 404
"""

# Write the content to the file
with open(os.path.join(tests_dir, 'test_tasks.py'), 'w') as f:
    f.write(test_file_content)

print(f"File '{os.path.join(tests_dir, 'test_tasks.py')}' created successfully.")

File 'tests/test_tasks.py' created successfully.
